In [ ]:
# Transformers installation
!pip install transformers datasets

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda"
model_id = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
encodings = tokenizer("The quick brown fox", return_tensors="pt").to(device)

In [ ]:
# Define the input text X and the corresponding output text Y
input_text = "What is the capital of France?"
output_text = "The capital of France is Paris."

# Tokenize the input and output texts
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = tokenizer.encode(output_text, return_tensors="pt")

# Concatenate the input and output token IDs TODO test
input_output_ids = torch.cat((input_ids, output_ids), dim=-1)
trg_len = len(output_ids)
target_ids = input_output_ids.clone()
target_ids[:, :-trg_len] = -100  #wtf does this do

In [ ]:
max_length = 1024
seq_len = encodings.input_ids.size(1)

nlls = []
end_loc = min(max_length, seq_len)
trg_len = end_loc # may be different from stride on last loop
input_ids = encodings.input_ids[:, 0:end_loc].to(device)
target_ids = input_ids.clone()
target_ids[:, :-trg_len] = -100  #wtf does this do

with torch.no_grad():
    outputs = model(input_ids, labels=target_ids)

    # loss is calculated using CrossEntropyLoss which averages over valid labels
    # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
    # to the left by 1.
    neg_log_likelihood = outputs.loss

nlls.append(neg_log_likelihood)    

ppl = torch.exp(torch.stack(nlls).mean())
print(neg_log_likelihood, ppl)

#Mass Calculation

In [1]:
import pandas as pd
# Read the CSV file into a DataFrame
df = pd.read_csv('prompts_completions.csv')

# Extract the prompts array from the DataFrame
prompts = df['prompt'].tolist()
outputs = df['output'].tolist()

# Create a new column in the DataFrame to store the generated outputs
df['perplexity'] = ''

for i, output in enumerate(outputs):
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Output: {generated_text!r}")

    # Save the generated output in the corresponding row of the new column
    df.at[i, 'mistral3'] = generated_text

# Save the updated DataFrame back to a CSV file
df.to_csv('prompts_completions.csv', index=False)

KeyError: 'output'